"""
c*number
The close price for the symbol in the given time period.

h*number
The highest price for the symbol in the given time period.

l*number
The lowest price for the symbol in the given time period.

ninteger
The number of transactions in the aggregate window.

o*number
The open price for the symbol in the given time period.

otcboolean
Whether or not this aggregate is for an OTC ticker. This field will be left off if false.

t*integer
The Unix Msec timestamp for the start of the aggregate window.

v*number
The trading volume of the symbol in the given time period.

vwnumber
The volume weighted average price.
"""

In [1]:
# import requests
# import pandas as pd
# from datetime import datetime
# def transform_date(x):
#     time_st = x / 1000
#     return datetime.utcfromtimestamp(time_st)

# # Replace with your actual Polygon.io API key
# API_KEY = "XSOZD_TVgvpUqvREF1FOqTQ2QbBpO8AG"

# # Parameters for the API call
# symbol = "AAPL"  # Stock symbol
# from_date = "2024-01-01"  # Start date in YYYY-MM-DD format
# to_date = "2024-11-11"  # End date in YYYY-MM-DD format
# interval = "minute"  # Interval: 'minute', 'hour', 'day', 'week', 'month'

# # Base URL for historical data
# base_url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/30/{interval}/{from_date}/{to_date}"

# # Make the API request
# response = requests.get(base_url, params={"apiKey": API_KEY})

# # Check the response status and handle the data
# if response.status_code == 200:
#     data = response.json()

# df = pd.DataFrame(data['results'])
# df.columns = ['Volume', 'Weighted Volume', 'Open', 'Close', 'High', 'Low', 'Timestamp', 'Num_trans']
# df['Timestamp'] = df['Timestamp'].apply(transform_date)

In [2]:
# if response.status_code == 200:
#     data = response.json()

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pathlib

def transform_date(a_timestamp):
    time_st = a_timestamp / 1000.0
    return datetime.utcfromtimestamp(time_st)

# Replace with your actual Polygon.io API key
API_KEY = "XSOZD_TVgvpUqvREF1FOqTQ2QbBpO8AG"
stocks = ["KKR",
"GM",
"SYF",
"CRWD",
"URI",
"DFS",
"ZBRA",
"TPR",
"MHK",
"FITB",
"RL"]
path = pathlib.Path(r'C:\Users\juann\Documents\Datasets')

# Parameters
start_date = datetime(2021, 2, 20)  # Start date
end_date = datetime(2024, 8, 30)  # End date
interval_desc = "minute"  # Interval: 'minute', 'hour', 'day', 'week', 'month'
interval_time = 30
limit = 50000  # Max records per API call

# Function to fetch data for a date range
def fetch_data(symbol, start_date, end_date, interval_time, interval_desc, limit, api_key):
    from_date = start_date.strftime('%Y-%m-%d')
    to_date = end_date.strftime('%Y-%m-%d')
    base_url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/{interval_time}/{interval_desc}/{from_date}/{to_date}"
    response = requests.get(base_url, params={"apiKey": api_key, "limit": limit})
    if response.status_code == 200:
        return response.json().get('results', [])
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

# Chunk date ranges to handle the limit
for symbol in stocks:
    all_data = []
    current_start = start_date
    while current_start < end_date:
        current_end = current_start + timedelta(days=7)  # 7-day chunks
        if current_end > end_date:
            current_end = end_date

        print(f"Fetching data from {current_start} to {current_end}")
        data = fetch_data(symbol, current_start, current_end, interval_time, interval_desc, limit, API_KEY)
        if data:
            all_data.extend(data)
        current_start = current_end + timedelta(days=1)  # Move to the next chunk

    # Convert to DataFrame
    if all_data:
        df = pd.DataFrame(all_data)
        df.columns = ['Volume', 'Weighted Volume', 'Open', 'Close', 'High', 'Low', 'Timestamp', 'Num_trans']
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms', utc=True)
        df['Timestamp'] = df['Timestamp'].dt.tz_convert('US/Eastern')
        df = df[(df['Timestamp'].dt.time >= pd.to_datetime("09:30").time()) &
                     (df['Timestamp'].dt.time <= pd.to_datetime("16:00").time())]
        df['Timestamp'] = df['Timestamp'].dt.tz_localize(None)
        
        
        print("Data fetching complete.")

        # Save or analyze the data
        file_name = f"{symbol}_{interval_time}_{interval_desc}_{start_date:%Y}_{end_date:%Y}.csv"
        file_path = path.joinpath(file_name)
        df.to_csv(file_path, index=False, sep=';')
    else:
        print("No data retrieved.")


Fetching data from 2021-02-20 00:00:00 to 2021-02-27 00:00:00
Fetching data from 2021-02-28 00:00:00 to 2021-03-07 00:00:00
Fetching data from 2021-03-08 00:00:00 to 2021-03-15 00:00:00
Fetching data from 2021-03-16 00:00:00 to 2021-03-23 00:00:00
Fetching data from 2021-03-24 00:00:00 to 2021-03-31 00:00:00
Fetching data from 2021-04-01 00:00:00 to 2021-04-08 00:00:00
Fetching data from 2021-04-09 00:00:00 to 2021-04-16 00:00:00
Fetching data from 2021-04-17 00:00:00 to 2021-04-24 00:00:00
Fetching data from 2021-04-25 00:00:00 to 2021-05-02 00:00:00
Fetching data from 2021-05-03 00:00:00 to 2021-05-10 00:00:00
Fetching data from 2021-05-11 00:00:00 to 2021-05-18 00:00:00
Fetching data from 2021-05-19 00:00:00 to 2021-05-26 00:00:00
Fetching data from 2021-05-27 00:00:00 to 2021-06-03 00:00:00
Fetching data from 2021-06-04 00:00:00 to 2021-06-11 00:00:00
Fetching data from 2021-06-12 00:00:00 to 2021-06-19 00:00:00
Fetching data from 2021-06-20 00:00:00 to 2021-06-27 00:00:00
Fetching

Fetching data from 2024-01-20 00:00:00 to 2024-01-27 00:00:00
Fetching data from 2024-01-28 00:00:00 to 2024-02-04 00:00:00
Fetching data from 2024-02-05 00:00:00 to 2024-02-12 00:00:00
Fetching data from 2024-02-13 00:00:00 to 2024-02-20 00:00:00
Fetching data from 2024-02-21 00:00:00 to 2024-02-28 00:00:00
Fetching data from 2024-02-29 00:00:00 to 2024-03-07 00:00:00
Fetching data from 2024-03-08 00:00:00 to 2024-03-15 00:00:00
Fetching data from 2024-03-16 00:00:00 to 2024-03-23 00:00:00
Fetching data from 2024-03-24 00:00:00 to 2024-03-31 00:00:00
Fetching data from 2024-04-01 00:00:00 to 2024-04-08 00:00:00
Fetching data from 2024-04-09 00:00:00 to 2024-04-16 00:00:00
Fetching data from 2024-04-17 00:00:00 to 2024-04-24 00:00:00
Fetching data from 2024-04-25 00:00:00 to 2024-05-02 00:00:00
Fetching data from 2024-05-03 00:00:00 to 2024-05-10 00:00:00
Fetching data from 2024-05-11 00:00:00 to 2024-05-18 00:00:00
Fetching data from 2024-05-19 00:00:00 to 2024-05-26 00:00:00
Fetching

Fetching data from 2023-06-02 00:00:00 to 2023-06-09 00:00:00
Fetching data from 2023-06-10 00:00:00 to 2023-06-17 00:00:00
Fetching data from 2023-06-18 00:00:00 to 2023-06-25 00:00:00
Fetching data from 2023-06-26 00:00:00 to 2023-07-03 00:00:00
Fetching data from 2023-07-04 00:00:00 to 2023-07-11 00:00:00
Fetching data from 2023-07-12 00:00:00 to 2023-07-19 00:00:00
Fetching data from 2023-07-20 00:00:00 to 2023-07-27 00:00:00
Fetching data from 2023-07-28 00:00:00 to 2023-08-04 00:00:00
Fetching data from 2023-08-05 00:00:00 to 2023-08-12 00:00:00
Fetching data from 2023-08-13 00:00:00 to 2023-08-20 00:00:00
Fetching data from 2023-08-21 00:00:00 to 2023-08-28 00:00:00
Fetching data from 2023-08-29 00:00:00 to 2023-09-05 00:00:00
Fetching data from 2023-09-06 00:00:00 to 2023-09-13 00:00:00
Fetching data from 2023-09-14 00:00:00 to 2023-09-21 00:00:00
Fetching data from 2023-09-22 00:00:00 to 2023-09-29 00:00:00
Fetching data from 2023-09-30 00:00:00 to 2023-10-07 00:00:00
Fetching

Fetching data from 2022-10-13 00:00:00 to 2022-10-20 00:00:00
Fetching data from 2022-10-21 00:00:00 to 2022-10-28 00:00:00
Fetching data from 2022-10-29 00:00:00 to 2022-11-05 00:00:00
Fetching data from 2022-11-06 00:00:00 to 2022-11-13 00:00:00
Fetching data from 2022-11-14 00:00:00 to 2022-11-21 00:00:00
Fetching data from 2022-11-22 00:00:00 to 2022-11-29 00:00:00
Fetching data from 2022-11-30 00:00:00 to 2022-12-07 00:00:00
Fetching data from 2022-12-08 00:00:00 to 2022-12-15 00:00:00
Fetching data from 2022-12-16 00:00:00 to 2022-12-23 00:00:00
Fetching data from 2022-12-24 00:00:00 to 2022-12-31 00:00:00
Fetching data from 2023-01-01 00:00:00 to 2023-01-08 00:00:00
Fetching data from 2023-01-09 00:00:00 to 2023-01-16 00:00:00
Fetching data from 2023-01-17 00:00:00 to 2023-01-24 00:00:00
Fetching data from 2023-01-25 00:00:00 to 2023-02-01 00:00:00
Fetching data from 2023-02-02 00:00:00 to 2023-02-09 00:00:00
Fetching data from 2023-02-10 00:00:00 to 2023-02-17 00:00:00
Fetching

Fetching data from 2022-02-23 00:00:00 to 2022-03-02 00:00:00
Fetching data from 2022-03-03 00:00:00 to 2022-03-10 00:00:00
Fetching data from 2022-03-11 00:00:00 to 2022-03-18 00:00:00
Fetching data from 2022-03-19 00:00:00 to 2022-03-26 00:00:00
Fetching data from 2022-03-27 00:00:00 to 2022-04-03 00:00:00
Fetching data from 2022-04-04 00:00:00 to 2022-04-11 00:00:00
Fetching data from 2022-04-12 00:00:00 to 2022-04-19 00:00:00
Fetching data from 2022-04-20 00:00:00 to 2022-04-27 00:00:00
Fetching data from 2022-04-28 00:00:00 to 2022-05-05 00:00:00
Fetching data from 2022-05-06 00:00:00 to 2022-05-13 00:00:00
Fetching data from 2022-05-14 00:00:00 to 2022-05-21 00:00:00
Fetching data from 2022-05-22 00:00:00 to 2022-05-29 00:00:00
Fetching data from 2022-05-30 00:00:00 to 2022-06-06 00:00:00
Fetching data from 2022-06-07 00:00:00 to 2022-06-14 00:00:00
Fetching data from 2022-06-15 00:00:00 to 2022-06-22 00:00:00
Fetching data from 2022-06-23 00:00:00 to 2022-06-30 00:00:00
Fetching

...

Fetching data from 2022-07-09 00:00:00 to 2022-07-16 00:00:00
Fetching data from 2022-07-17 00:00:00 to 2022-07-24 00:00:00
Fetching data from 2022-07-25 00:00:00 to 2022-08-01 00:00:00
Fetching data from 2022-08-02 00:00:00 to 2022-08-09 00:00:00
Fetching data from 2022-08-10 00:00:00 to 2022-08-17 00:00:00
Fetching data from 2022-08-18 00:00:00 to 2022-08-25 00:00:00
Fetching data from 2022-08-26 00:00:00 to 2022-09-02 00:00:00
Fetching data from 2022-09-03 00:00:00 to 2022-09-10 00:00:00
Fetching data from 2022-09-11 00:00:00 to 2022-09-18 00:00:00
Fetching data from 2022-09-19 00:00:00 to 2022-09-26 00:00:00
Fetching data from 2022-09-27 00:00:00 to 2022-10-04 00:00:00
Fetching data from 2022-10-05 00:00:00 to 2022-10-12 00:00:00
Fetching data from 2022-10-13 00:00:00 to 2022-10-20 00:00:00
Fetching data from 2022-10-21 00:00:00 to 2022-10-28 00:00:00
Fetching data from 2022-10-29 00:00:00 to 2022-11-05 00:00:00
Fetching data from 2022-11-06 00:00:00 to 2022-11-13 00:00:00
Fetching

Fetching data from 2021-11-19 00:00:00 to 2021-11-26 00:00:00
Fetching data from 2021-11-27 00:00:00 to 2021-12-04 00:00:00
Fetching data from 2021-12-05 00:00:00 to 2021-12-12 00:00:00
Fetching data from 2021-12-13 00:00:00 to 2021-12-20 00:00:00
Fetching data from 2021-12-21 00:00:00 to 2021-12-28 00:00:00
Fetching data from 2021-12-29 00:00:00 to 2022-01-05 00:00:00
Fetching data from 2022-01-06 00:00:00 to 2022-01-13 00:00:00
Fetching data from 2022-01-14 00:00:00 to 2022-01-21 00:00:00
Fetching data from 2022-01-22 00:00:00 to 2022-01-29 00:00:00
Fetching data from 2022-01-30 00:00:00 to 2022-02-06 00:00:00
Fetching data from 2022-02-07 00:00:00 to 2022-02-14 00:00:00
Fetching data from 2022-02-15 00:00:00 to 2022-02-22 00:00:00
Fetching data from 2022-02-23 00:00:00 to 2022-03-02 00:00:00
Fetching data from 2022-03-03 00:00:00 to 2022-03-10 00:00:00
Fetching data from 2022-03-11 00:00:00 to 2022-03-18 00:00:00
Fetching data from 2022-03-19 00:00:00 to 2022-03-26 00:00:00
Fetching

Fetching data from 2021-04-01 00:00:00 to 2021-04-08 00:00:00
Fetching data from 2021-04-09 00:00:00 to 2021-04-16 00:00:00
Fetching data from 2021-04-17 00:00:00 to 2021-04-24 00:00:00
Fetching data from 2021-04-25 00:00:00 to 2021-05-02 00:00:00
Fetching data from 2021-05-03 00:00:00 to 2021-05-10 00:00:00
Fetching data from 2021-05-11 00:00:00 to 2021-05-18 00:00:00
Fetching data from 2021-05-19 00:00:00 to 2021-05-26 00:00:00
Fetching data from 2021-05-27 00:00:00 to 2021-06-03 00:00:00
Fetching data from 2021-06-04 00:00:00 to 2021-06-11 00:00:00
Fetching data from 2021-06-12 00:00:00 to 2021-06-19 00:00:00
Fetching data from 2021-06-20 00:00:00 to 2021-06-27 00:00:00
Fetching data from 2021-06-28 00:00:00 to 2021-07-05 00:00:00
Fetching data from 2021-07-06 00:00:00 to 2021-07-13 00:00:00
Fetching data from 2021-07-14 00:00:00 to 2021-07-21 00:00:00
Fetching data from 2021-07-22 00:00:00 to 2021-07-29 00:00:00
Fetching data from 2021-07-30 00:00:00 to 2021-08-06 00:00:00
Fetching

Fetching data from 2024-02-29 00:00:00 to 2024-03-07 00:00:00
Fetching data from 2024-03-08 00:00:00 to 2024-03-15 00:00:00
Fetching data from 2024-03-16 00:00:00 to 2024-03-23 00:00:00
Fetching data from 2024-03-24 00:00:00 to 2024-03-31 00:00:00
Fetching data from 2024-04-01 00:00:00 to 2024-04-08 00:00:00
Fetching data from 2024-04-09 00:00:00 to 2024-04-16 00:00:00
Fetching data from 2024-04-17 00:00:00 to 2024-04-24 00:00:00
Fetching data from 2024-04-25 00:00:00 to 2024-05-02 00:00:00
Fetching data from 2024-05-03 00:00:00 to 2024-05-10 00:00:00
Fetching data from 2024-05-11 00:00:00 to 2024-05-18 00:00:00
Fetching data from 2024-05-19 00:00:00 to 2024-05-26 00:00:00
Fetching data from 2024-05-27 00:00:00 to 2024-06-03 00:00:00
Fetching data from 2024-06-04 00:00:00 to 2024-06-11 00:00:00
Fetching data from 2024-06-12 00:00:00 to 2024-06-19 00:00:00
Fetching data from 2024-06-20 00:00:00 to 2024-06-27 00:00:00
Fetching data from 2024-06-28 00:00:00 to 2024-07-05 00:00:00
Fetching

Fetching data from 2023-07-12 00:00:00 to 2023-07-19 00:00:00
Fetching data from 2023-07-20 00:00:00 to 2023-07-27 00:00:00
Fetching data from 2023-07-28 00:00:00 to 2023-08-04 00:00:00
Fetching data from 2023-08-05 00:00:00 to 2023-08-12 00:00:00
Fetching data from 2023-08-13 00:00:00 to 2023-08-20 00:00:00
Fetching data from 2023-08-21 00:00:00 to 2023-08-28 00:00:00
Fetching data from 2023-08-29 00:00:00 to 2023-09-05 00:00:00
Fetching data from 2023-09-06 00:00:00 to 2023-09-13 00:00:00
Fetching data from 2023-09-14 00:00:00 to 2023-09-21 00:00:00
Fetching data from 2023-09-22 00:00:00 to 2023-09-29 00:00:00
Fetching data from 2023-09-30 00:00:00 to 2023-10-07 00:00:00
Fetching data from 2023-10-08 00:00:00 to 2023-10-15 00:00:00
Fetching data from 2023-10-16 00:00:00 to 2023-10-23 00:00:00
Fetching data from 2023-10-24 00:00:00 to 2023-10-31 00:00:00
Fetching data from 2023-11-01 00:00:00 to 2023-11-08 00:00:00
Fetching data from 2023-11-09 00:00:00 to 2023-11-16 00:00:00
Fetching

In [4]:
df

,Volume,Weighted Volume,Open,Close,High,Low,Timestamp,Num_trans
11,17766784.0,152.3331,150.36,154.0900,154.1400,149.9800,2024-01-16 09:30:00,159564
12,12596526.0,155.6487,154.10,157.1477,157.5000,153.5200,2024-01-16 10:00:00,106516
13,12107292.0,158.4521,157.14,159.3738,159.7150,157.0000,2024-01-16 10:30:00,96391
14,10450130.0,158.4074,159.38,158.7100,159.5800,157.3800,2024-01-16 11:00:00,87357
15,7515840.0,158.9975,158.72,158.2800,159.5100,157.9300,2024-01-16 11:30:00,56652
...,...,...,...,...,...,...,...,...
6824,992574.0,139.9422,140.51,139.5500,140.5264,139.4750,2024-11-18 14:00:00,12684
6825,983093.0,139.7910,139.55,139.9077,140.0300,139.4500,2024-11-18 14:30:00,13070
6826,1639910.0,139.8255,139.90,139.8350,139.9600,139.5800,2024-11-18 15:00:00,13833
6827,3132279.0,139.1518,139.83,138.9800,139.8300,138.6301,2024-11-18 15:30:00,39962


In [55]:
df.to_csv('tst.csv')

In [19]:
# Convert UTC timestamps to local time zone
local_timezone = pytz.timezone('America/New_York')  # Example: Convert to New York time zone

# df['timestamp_local'] = df['timestamp_utc'].dt.tz_localize('UTC')

pd.to_datetime(df['Timestamp'],unit='ms').dt.tz_convert(local_timezone)

NameError: name 'pytz' is not defined

In [22]:
df['test'] = pd.to_datetime(df['Timestamp'], unit='ms', utc=True)

df['test'] = df['test'].dt.tz_convert('US/Eastern')

In [23]:
df

,Volume,Weighted Volume,Open,Close,High,Low,Timestamp,Num_trans,test
0,2315.0,208.9400,208.76,209.11,209.11,208.76,1731315600000,110,2024-11-11 04:00:00-05:00
1,2969.0,208.9455,209.01,208.75,209.01,208.75,1731315660000,51,2024-11-11 04:01:00-05:00
2,819.0,208.8574,208.86,208.82,208.86,208.82,1731315720000,51,2024-11-11 04:02:00-05:00
3,1311.0,208.7206,208.66,208.75,208.75,208.66,1731315900000,33,2024-11-11 04:05:00-05:00
4,1045.0,208.8505,208.84,208.90,208.91,208.84,1731315960000,30,2024-11-11 04:06:00-05:00
...,...,...,...,...,...,...,...,...,...
2243,1314.0,214.9200,214.87,214.87,214.87,214.87,1731576000000,70,2024-11-14 04:20:00-05:00
2244,705.0,214.8366,214.83,214.83,214.83,214.83,1731576060000,16,2024-11-14 04:21:00-05:00
2245,1790.0,214.7659,214.77,214.77,214.77,214.77,1731576180000,50,2024-11-14 04:23:00-05:00
2246,684.0,214.8836,214.89,214.89,214.89,214.89,1731576360000,27,2024-11-14 04:26:00-05:00


In [13]:
# import yfinance as yf
# import pandas as pd
# import talib

# # Define function to calculate similarity
# def find_similar_stocks(stock_list, min_volume=1_000_000, min_volatility=0.02):
#     """
#     Identifies stocks with similar characteristics to the given stock list.
    
#     Parameters:
#         stock_list (list): List of ticker symbols.
#         min_volume (int): Minimum average daily volume.
#         min_volatility (float): Minimum daily price movement as a percentage.
    
#     Returns:
#         DataFrame: Stocks matching the criteria with key metrics.
#     """
#     similar_stocks = []

#     for stock in stock_list:
#         # Fetch stock data
#         ticker = yf.Ticker(stock)
#         data = ticker.history(period="1y")
        
#         # Calculate metrics
#         avg_volume = data['Volume'].mean()
#         data['Daily Change'] = (data['High'] - data['Low']) / data['Low']
#         avg_volatility = data['Daily Change'].mean()
        
#         # Example technical indicators
#         close_prices = data['Close'].values
#         rsi = talib.RSI(close_prices, timeperiod=14).mean()  # Average RSI
        
#         # Add to similar stocks if criteria are met
#         if avg_volume >= min_volume and avg_volatility >= min_volatility and rsi > 50:
#             similar_stocks.append({
#                 "Ticker": stock,
#                 "Average Volume": avg_volume,
#                 "Average Volatility": avg_volatility,
#                 "RSI": rsi
#             })
    
#     return pd.DataFrame(similar_stocks)

# # Input list of known stocks
# stock_list = ["AAPL", "MSFT", "TSLA", "AMZN", "JNJ", "GOOGL", "NFLX"]

# # Find similar stocks
# similar_stocks_df = find_similar_stocks(stock_list)

# # Display the results
# print(similar_stocks_df)


Empty DataFrame
Columns: []
Index: []
